In [ ]:
!pip install SQLAlchemy

In [ ]:
!pip install psycopg2

In [10]:
import sqlalchemy
import pandas as pd
import numpy as np
from geopy import distance
from tqdm import tqdm

from sqlalchemy import create_engine
engine = create_engine('postgresql+psycopg2://dairul.fuhron@gudangada.com:NmgNkFA53wepusP9@bi-playground.c6l2qlyxjdnr.ap-southeast-1.rds.amazonaws.com:5432/bi-playground')
query_anchor = """
SELECT      s.id store_id
            , s.unique_id
            , aa.name city 
            , address_latitude
            , address_longitude
FROM        "gada-marketplace".store s
LEFT JOIN   "gada-marketplace".address_area aa ON s.address_city_id = aa.id AND aa.type = 'CITY'
LEFT JOIN   "gada-marketplace".fee_group fg ON s.fee_group_id = fg.id
WHERE       s.deleted IS NULL
            AND s.is_active IS TRUE
            AND allows_selling IS FALSE
            AND acquisition_channel = 'NON_REFERRED'
            AND fg.name <> 'TESTING'
            AND s.store_name NOT LIKE '%.'
"""
query_sup = """
SELECT      s.id store_id
            , s.unique_id
            , aa.name city 
            , address_latitude
            , address_longitude
FROM        "gada-marketplace".store s
LEFT JOIN   "gada-marketplace".address_area aa ON s.address_city_id = aa.id AND aa.type = 'CITY'
LEFT JOIN   "gada-marketplace".fee_group fg ON s.fee_group_id = fg.id
JOIN        (
            SELECT    DISTINCT store_id
            FROM      "gada-marketplace".inventory i
            WHERE     i.deleted IS NULL
                      AND i.is_active IS TRUE
                      AND i.available_stock > 0
            ) x ON s.id = x.store_id
WHERE       s.deleted IS NULL
            AND s.is_active IS TRUE
            AND allows_selling IS TRUE
            AND fg.name <> 'TESTING'
"""
df_anchor = pd.read_sql_query(query_anchor, engine)
df_sup = pd.read_sql_query(query_sup, engine)
# print(df_anchor)
# print(df_sup)
df_anchor.info()
df_sup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13164 entries, 0 to 13163
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   store_id           13164 non-null  int64  
 1   unique_id          13164 non-null  object 
 2   city               13146 non-null  object 
 3   address_latitude   13164 non-null  float64
 4   address_longitude  13164 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 514.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6102 entries, 0 to 6101
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   store_id           6102 non-null   int64  
 1   unique_id          6102 non-null   object 
 2   city               6101 non-null   object 
 3   address_latitude   6102 non-null   float64
 4   address_longitude  6102 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usa

In [11]:
# For getting 15 KM Radius from each Anchor to Support(s)

lst = []
group_id = 1
for x,y,z in tqdm(df_anchor[['unique_id', 'address_latitude', 'address_longitude']].itertuples(index=False)):
    for a,b,c in df_sup[['unique_id', 'address_latitude', 'address_longitude']].itertuples(index=False):
        coord_anchor = (y, z)
        coord_support = (b, c)
        radius = (distance.distance(coord_anchor,coord_support).km)
        if radius <= 15:
            lst.append({'group_id': group_id, 
                        'anchor': x, 
                        'support':a,
                        'support_latitude': b,
                        'support_longitude': c,
                        'phase_loop': 1, 
                        'radius': radius
                       })
    group_id+=1
df_1 = pd.DataFrame(lst)
print(df_1)

13164it [5:09:04,  1.41s/it]


        group_id         anchor  ... phase_loop     radius
0              1  GADA-gajlxjkz  ...          1  13.926164
1              3  GADA-d0raqmpk  ...          1  14.249137
2              3  GADA-d0raqmpk  ...          1   4.155910
3              3  GADA-d0raqmpk  ...          1   7.943419
4              3  GADA-d0raqmpk  ...          1  14.674956
...          ...            ...  ...        ...        ...
838228     13164  GADA-zxao4m5g  ...          1   7.791152
838229     13164  GADA-zxao4m5g  ...          1  13.832303
838230     13164  GADA-zxao4m5g  ...          1  14.043833
838231     13164  GADA-zxao4m5g  ...          1   7.496625
838232     13164  GADA-zxao4m5g  ...          1   6.336232

[838233 rows x 7 columns]


In [12]:
df_1.to_sql('irul_online_buyer_seller_density', con=engine, schema='public', if_exists='replace',
           index=False, method='multi')